In [43]:
!pip install transformers torch scikit-learn pandas

In [44]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

# Cấu hình thiết bị
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [45]:
df = pd.read_csv("/kaggle/input/tiktok-comments/tiktok_comments_balanced.csv")
df.head()

,line_number,text,emotion_label
0,2679,sợ thật,3
1,26129,ối dồi ôi fpt shop cháy tới đó mệt_á,2
2,20113,xem mà khóc thương_k chịu dc,2
3,17380,đàn_bà sống thọ hơn đàn_ông là vậy k,0
4,18200,cô ấy già đi nhiều quá vẫn nhớ ảnh chụp cô lúc...,2


In [46]:
label_mapping = {
    0: "Vui vẻ",        # tích cực
    1: "Tức giận",      # negative (giận dữ, bực tức)
    2: "Buồn",          # sadness
    3: "Sợ hãi",        # lo lắng, hoảng loạn
    4: "Trung lập"      # neutral
}
df["label_text"] = df["emotion_label"].map(label_mapping)

le = LabelEncoder()
df['label'] = le.fit_transform(df['label_text'])
num_classes = len(le.classes_)


df

,line_number,text,emotion_label,label_text,label
0,2679,sợ thật,3,Sợ hãi,1
1,26129,ối dồi ôi fpt shop cháy tới đó mệt_á,2,Buồn,0
2,20113,xem mà khóc thương_k chịu dc,2,Buồn,0
3,17380,đàn_bà sống thọ hơn đàn_ông là vậy k,0,Vui vẻ,4
4,18200,cô ấy già đi nhiều quá vẫn nhớ ảnh chụp cô lúc...,2,Buồn,0
...,...,...,...,...,...
14448,26549,nhìn sợ thế,3,Sợ hãi,1
14449,19664,khổ thân hai nhà bên cạnh,2,Buồn,0
14450,27967,hãi lun vk ơi,3,Sợ hãi,1
14451,7682,vũ,4,Trung lập,2


In [47]:
# Chia dữ liệu
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'], df['label'], test_size=0.2, stratify=df['label'], random_state=42
)

# Tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
MAX_LEN = 64

def tokenize(texts):
    input_ids, attention_masks = [], []
    for text in texts:
        encoded = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=MAX_LEN,
            truncation=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt'
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    return torch.cat(input_ids, dim=0), torch.cat(attention_masks, dim=0)

# Token hóa
train_ids, train_masks = tokenize(train_texts.tolist())
val_ids, val_masks = tokenize(val_texts.tolist())

train_labels = torch.tensor(train_labels.tolist())
val_labels = torch.tensor(val_labels.tolist())

# DataLoader
train_data = TensorDataset(train_ids, train_masks, train_labels)
val_data = TensorDataset(val_ids, val_masks, val_labels)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32)

In [48]:
# Load BERT và LSTM
bert = BertModel.from_pretrained("bert-base-multilingual-cased").to(device)
bert.eval() 

# Định nghĩa LSTM classifier thủ tục
lstm = nn.LSTM(input_size=768, hidden_size=128, num_layers=1,
               batch_first=True, bidirectional=True).to(device)
dropout = nn.Dropout(0.3).to(device)
fc = nn.Linear(128 * 2, num_classes).to(device)

params = list(lstm.parameters()) + list(fc.parameters())
optimizer = optim.Adam(params, lr=2e-4)
criterion = nn.CrossEntropyLoss()

In [49]:
# Huấn luyện
EPOCHS = 10
for epoch in range(EPOCHS):
    lstm.train()
    fc.train()
    total_loss = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]

        with torch.no_grad():
            bert_output = bert(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = bert_output.last_hidden_state  # (B, T, 768)

        lstm_out, _ = lstm(sequence_output)
        final_output = lstm_out[:, -1, :]
        dropped = dropout(final_output)
        logits = fc(dropped)

        loss = criterion(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1} - Loss: {total_loss/len(train_loader):.4f}")

Epoch 1: 100%|██████████| 362/362 [00:43<00:00,  8.40it/s]


Epoch 1 - Loss: 1.3652


Epoch 2: 100%|██████████| 362/362 [00:43<00:00,  8.34it/s]


Epoch 2 - Loss: 1.1975


Epoch 3: 100%|██████████| 362/362 [00:42<00:00,  8.50it/s]


...

Epoch 8 - Loss: 0.8613


Epoch 9: 100%|██████████| 362/362 [00:43<00:00,  8.41it/s]


Epoch 9 - Loss: 0.8158


Epoch 10: 100%|██████████| 362/362 [00:43<00:00,  8.41it/s]

Epoch 10 - Loss: 0.7595


**Đánh giá**

In [50]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import numpy as np

lstm.eval()
fc.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(val_loader, desc="Đánh giá"):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]

        bert_output = bert(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = bert_output.last_hidden_state

        lstm_out, _ = lstm(sequence_output)
        final_output = lstm_out[:, -1, :]
        logits = fc(dropout(final_output))

        preds = torch.argmax(logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Đánh giá
acc = accuracy_score(all_labels, all_preds)
print(f"\n✅ Accuracy: {acc:.4f}\n")

print("📋 Classification Report:")
print(classification_report(all_labels, all_preds, target_names=le.classes_.astype(str)))

# (Tuỳ chọn) Confusion Matrix
print("📊 Confusion Matrix:")
print(confusion_matrix(all_labels, all_preds))


Đánh giá: 100%|██████████| 91/91 [00:10<00:00,  8.68it/s]


✅ Accuracy: 0.5358

📋 Classification Report:
              precision    recall  f1-score   support

        Buồn       0.53      0.52      0.53       626
      Sợ hãi       0.82      0.60      0.69       551
   Trung lập       0.50      0.61      0.55       551
    Tức giận       0.56      0.48      0.51       611
      Vui vẻ       0.39      0.48      0.43       552

    accuracy                           0.54      2891
   macro avg       0.56      0.54      0.54      2891
weighted avg       0.56      0.54      0.54      2891

📊 Confusion Matrix:
[[326  14  85  76 125]
 [ 68 331  50  52  50]
 [ 45  10 336  40 120]
 [ 92  34  71 291 123]
 [ 80  17 129  61 265]]


Test model

In [51]:
def predict_sentiment(texts):
    if isinstance(texts, str):
        texts = [texts]

    # Token hóa
    input_ids, attention_masks = tokenize(texts)
    input_ids, attention_masks = input_ids.to(device), attention_masks.to(device)

    # Trích đặc trưng từ BERT
    with torch.no_grad():
        bert_output = bert(input_ids=input_ids, attention_mask=attention_masks)
        sequence_output = bert_output.last_hidden_state

        lstm_out, _ = lstm(sequence_output)
        final_output = lstm_out[:, -1, :]
        logits = fc(dropout(final_output))
        preds = torch.argmax(logits, dim=1)

    return le.inverse_transform(preds.cpu().numpy())


In [52]:
test_inputs = [
    "Tự nhiên ra đường cái k đụng chạm j ai củng sợ ngang😑",
    "Lúc đầu thì ""Tùng Bò đâm, xong 1 lúc sau thì ""vào tù nha""😂",
    "bắt quá nhanh 🥰 Công An Việt Nam mình quá giỏi, xuất sắc 🥰"
]

predicted_labels = predict_sentiment(test_inputs)

for text, label in zip(test_inputs, predicted_labels):
    print(f"📝 \"{text}\" → 💬 Dự đoán: {label}")


📝 "Tự nhiên ra đường cái k đụng chạm j ai củng sợ ngang😑" → 💬 Dự đoán: Sợ hãi
📝 "Lúc đầu thì Tùng Bò đâm, xong 1 lúc sau thì vào tù nha😂" → 💬 Dự đoán: Tức giận
📝 "bắt quá nhanh 🥰 Công An Việt Nam mình quá giỏi, xuất sắc 🥰" → 💬 Dự đoán: Vui vẻ
